In [6]:
# import libraries
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest

In [7]:
AZURE_COGNITIVE_ENDPOINT = "azure service endpoint"
AZURE_COGNITIVE_KEY = "azure service key"

# Step 1: azure Document Intelligence

Input: a pdf n

Output: location of identified text, table and figures with corresponding text

In [10]:
# helper functions

def get_words(page, line):
    result = []
    for word in page.words:
        if _in_span(word, line.spans):
            result.append(word)
    return result


def _in_span(word, spans):
    for span in spans:
        if word.span.offset >= span.offset and (
            word.span.offset + word.span.length
        ) <= (span.offset + span.length):
            return True
    return False


def analyze_layout():
    # sample document
    formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"

    document_intelligence_client = DocumentIntelligenceClient(
        endpoint=AZURE_COGNITIVE_ENDPOINT, credential=AzureKeyCredential(AZURE_COGNITIVE_KEY)
    )

    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", AnalyzeDocumentRequest(url_source=formUrl
    ))

    result: AnalyzeResult = poller.result()

    if result.styles and any([style.is_handwritten for style in result.styles]):
        print("Document contains handwritten content")
    else:
        print("Document does not contain handwritten content")

    for page in result.pages:
        print(f"----Analyzing layout from page #{page.page_number}----")
        print(
            f"Page has width: {page.width} and height: {page.height}, measured with unit: {page.unit}"
        )

        if page.lines:
            for line_idx, line in enumerate(page.lines):
                words = get_words(page, line)
                print(
                    f"...Line # {line_idx} has word count {len(words)} and text '{line.content}' "
                    f"within bounding polygon '{line.polygon}'"
                )

                for word in words:
                    print(
                        f"......Word '{word.content}' has a confidence of {word.confidence}"
                    )

        if page.selection_marks:
            for selection_mark in page.selection_marks:
                print(
                    f"Selection mark is '{selection_mark.state}' within bounding polygon "
                    f"'{selection_mark.polygon}' and has a confidence of {selection_mark.confidence}"
                )

    if result.tables:
        for table_idx, table in enumerate(result.tables):
            print(
                f"Table # {table_idx} has {table.row_count} rows and "
                f"{table.column_count} columns"
            )
            if table.bounding_regions:
                for region in table.bounding_regions:
                    print(
                        f"Table # {table_idx} location on page: {region.page_number} is {region.polygon}"
                    )
            for cell in table.cells:
                print(
                    f"...Cell[{cell.row_index}][{cell.column_index}] has text '{cell.content}'"
                )
                if cell.bounding_regions:
                    for region in cell.bounding_regions:
                        print(
                            f"...content on page {region.page_number} is within bounding polygon '{region.polygon}'"
                        )

    print("----------------------------------------")


if __name__ == "__main__":
    analyze_layout()

Document does not contain handwritten content
----Analyzing layout from page #1----
Page has width: 8.5 and height: 11, measured with unit: LengthUnit.INCH
...Line # 0 has word count 2 and text 'UNITED STATES' within bounding polygon '[3.4695, 0.6555, 5.0216, 0.6576, 5.0214, 0.847, 3.4693, 0.845]'
......Word 'UNITED' has a confidence of 0.997
......Word 'STATES' has a confidence of 0.998
...Line # 1 has word count 4 and text 'SECURITIES AND EXCHANGE COMMISSION' within bounding polygon '[2.1754, 0.8727, 6.3155, 0.8723, 6.3155, 1.0737, 2.1754, 1.0742]'
......Word 'SECURITIES' has a confidence of 0.993
......Word 'AND' has a confidence of 0.998
......Word 'EXCHANGE' has a confidence of 0.992
......Word 'COMMISSION' has a confidence of 0.992
...Line # 2 has word count 3 and text 'Washington, D.C. 20549' within bounding polygon '[3.4515, 1.0922, 5.0382, 1.0888, 5.0386, 1.2532, 3.4518, 1.2565]'
......Word 'Washington,' has a confidence of 0.995
......Word 'D.C.' has a confidence of 0.993
...

...content on page 1 is within bounding polygon '[3.8389, 5.1255, 5.3713, 5.1255, 5.3713, 5.2962, 3.8389, 5.2962]'
...Cell[1][2] has text 'NASDAQ'
...content on page 1 is within bounding polygon '[5.3713, 5.1255, 6.93, 5.1123, 6.93, 5.2962, 5.3713, 5.2962]'
...Cell[2][0] has text '2.125% Notes due 2021'
...content on page 1 is within bounding polygon '[0.5636, 5.2962, 3.8389, 5.2962, 3.8323, 5.4538, 0.5636, 5.4538]'
...Cell[2][1] has text 'MSFT'
...content on page 1 is within bounding polygon '[3.8389, 5.2962, 5.3713, 5.2962, 5.3713, 5.4538, 3.8323, 5.4538]'
...Cell[2][2] has text 'NASDAQ'
...content on page 1 is within bounding polygon '[5.3713, 5.2962, 6.93, 5.2962, 6.93, 5.4538, 5.3713, 5.4538]'
...Cell[3][0] has text '3.125% Notes due 2028'
...content on page 1 is within bounding polygon '[0.5636, 5.4538, 3.8323, 5.4538, 3.8323, 5.6114, 0.5636, 5.6114]'
...Cell[3][1] has text 'MSFT'
...content on page 1 is within bounding polygon '[3.8323, 5.4538, 5.3713, 5.4538, 5.3713, 5.6114, 3.

## Step 2: Azure OpenAI Studio 